# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import sys
!{sys.executable} -m pip install gmaps
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from g import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_df = pd.read_csv('city_weather_data.csv')
city_df.head()

,city,latitude,longitude,temperature,humidity,wind_speed,clouds
0,Ushuaia,-54.8000,-68.3000,49.66,46,20.71,75
1,Dikson,73.5069,80.5464,27.05,62,13.15,99
2,Dalvik,65.9702,-18.5286,26.53,77,8.01,36
3,Qaanaaq,77.4840,-69.3632,12.74,83,5.23,94
4,Cape Town,-33.9258,18.4232,66.13,72,3.00,0


In [3]:
city_df.dtypes

city            object
latitude       float64
longitude      float64
temperature    float64
humidity         int64
wind_speed     float64
clouds           int64
dtype: object

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=gkey)
locations = city_df[['latitude', 'longitude']]
humidity = city_df['humidity']

In [5]:
fig = gmaps.figure()

humidity_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
fig.add_layer(heatmap_layer)

fig

NameError: name 'heatmap_layer' is not defined

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
ideal_df=city_df[(city_df['temperature']>70) & (city_df['temperature']<80)]
ideal_df['temperature'].describe()
ideal_df = ideal_df[ideal_df['wind_speed']<10]
ideal_df['wind_speed'].describe()
ideal_df = ideal_df[ideal_df['clouds']==0]
ideal_df.head(10)

,city,latitude,longitude,temperature,humidity,wind_speed,clouds
151,Sheopur,25.6667,76.7000,73.45,42,4.27,0
302,Algiers,36.7525,3.0420,71.42,60,5.75,0
419,Bharatpur,27.2167,77.4833,77.13,42,1.21,0
476,Seondha,26.1578,78.7797,75.18,51,4.18,0
606,Dalbandin,28.8947,64.4101,76.57,13,3.83,0
625,Budhlada,29.9333,75.5667,74.48,30,3.15,0
704,Anta,25.1500,76.3000,74.19,40,6.02,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
ideal_df['Hotel Name']= ''
ideal_df

,city,latitude,longitude,temperature,humidity,wind_speed,clouds,Hotel Name
151,Sheopur,25.6667,76.7000,73.45,42,4.27,0,
302,Algiers,36.7525,3.0420,71.42,60,5.75,0,
419,Bharatpur,27.2167,77.4833,77.13,42,1.21,0,
476,Seondha,26.1578,78.7797,75.18,51,4.18,0,
606,Dalbandin,28.8947,64.4101,76.57,13,3.83,0,
625,Budhlada,29.9333,75.5667,74.48,30,3.15,0,
704,Anta,25.1500,76.3000,74.19,40,6.02,0,


In [9]:
target_search = "Hotel"
coords = ideal_df[['latitude', 'longitude']
radius = 5000
url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
hotel_name = ideal_df['Hotel Name'].tolist()
# redefine params
params = {
    "location": coords,
    "keyword": target_search,
    "radius": radius,
    "key": gkey
}

                  
                  

SyntaxError: invalid syntax (<ipython-input-9-138ba4bef8c0>, line 3)

In [ ]:
for index, row in ideal_df.iterrows():
                  response = requests.get(url, params=params).json()
                  hotel_name.append(response)

In [ ]:
hotel = gmaps.symbol_layer(
    locations, 
    info_box_content=[f'Hotel: {Hotel Name}' for Hotel in hotel_name]
)

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map


# Display figure
fig = gmaps.figure()

fig.add_layer(humidity_layer)
fig.add_layer(hotel)

fig

NameError: name 'hotel' is not defined